<a href="https://colab.research.google.com/github/FaridFK/pola-batik-baru/blob/main/batik_GAN_coba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --no-cache-dir gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [2]:
import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def show(tensor, num=25, wandbactive=0, name=''):

  # mengambil data tensor yang didetach terlebih dahulu dari gpu dan ditransfer ke cpu agar dapat ditampilkan oleh fungsi plt.imshow

  data = tensor.detach().cpu()

  # membuat grid dari data yang sudah diambil dari tensor, dengan baris 5.
  # lalu dipermute(1, 2, 0) karena data dari tensor sebelumnya memiliki dimensi(batch_size, channel, height, width).
  # setelah dipermute menjadi (heigh, width, channel). hasil dari permute membuat data dapat ditampilkan dengan benar menggunakan plt.imshow

  grid = make_grid(data[:num], nrow=5).permute(1,2,0)

  # optional
  # apabila wandbact=1 mengirim grid ke wandb sesuai dengan parameter input name

  if(wandbactive==1):
    wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})
  plt.imshow(grid.clip(0,1))
  plt.show()

# hyperparameter & general parameter
# jumlah epoch

n_epochs = 10000

# jumlah sample yang digunakan dalam satu iterasi learning

batch_size = 128

# learning rate = 0.0001

lr = 1e-4

# dimensi ruang laten yang dipakai dalam generator (jumlah fitur yang digunakan untuk mengambil sampel acak dari ruang laten)
z_dim = 200

device = 'cuda'
cur_step = 0
crit_cycles = 5
gen_losses = []
crit_losses = []
show_step = 35
save_step = 35
wandbact = 1

In [3]:
# optional
!pip install wandb -qqq
import wandb
wandb.login(key='444a4cc88c98ea16d3933f474a957b7a3e10390f')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%%capture
experiment_name = "wgan"
myrun = wandb.init(
    project = "wgan",
    group = experiment_name,
    config = {
        "optimizer":"adam",
        "model":"wgan gp",
        "epoch":"1000",
        "batch_size":128
    }
)
config = wandb.config

In [5]:
print(experiment_name)

wgan


In [6]:
from torch.nn.modules.activation import Tanh
from torch.nn.modules.conv import Conv2d

# generator model
# mendefinisikan kelas generator yang mewarisi kelas nn.Module dari pytorch, digunakannya nn.Module dikarenakan mempunyai fungsi pembantu
# yaitu forward propagation dan backward propagation, serta parameter2 untuk menyimpan nilai layer dalam model

class Generator(nn.Module):

  # mendefinisikan fungsi init sebagai konstruktor kelas generator dengan parameter z_dim dan d_dim
  # z_dim : jumlah ruang laten yang digunakan sebagai input generator
  # d_dim : d_dim adalah jumlah channel yang digunakan dalam setiap lapisan (layer)

  def __init__(self, z_dim=64, d_dim=16):
    super(Generator, self).__init__()
    self.z_dim = z_dim

    # diinisasi gen dengan nilai perhitungan melalui nn.Sequential
    # nn.Sequential adalah modul yang berfungsi untuk mengelompokan modul modul menjadi rangkaian modul yang lebih besar.
    # dimana modul yang dirangkai akan dieksekusi secara berurutan.
    # dengan setiap modul akan menerima input dari modul sebelumnya dan menghasilkan -
    # output yang akan digunakan sebagai input untuk modul berikutnya

    self.gen = nn.Sequential(

        # ConvTranspose2d : in_channels, out_channels, kernel_size, stride=1, padding=0
        # calculating new width & height : (n - 1) * stride - 2 * padding + ks
        # n : height & width, ks : kernel size
        # begin with 1x1 image with z_dim number of channels (200)

        # Conv2Transpose2d merupakan operasi yang digunakan untuk konvolusi transpose atau de-convolution pada tensor.
        # modul ini memiliki beberapa parameter yaitu :
        # in_channels : jumlah input channel
        # out_channels : jumlah output channel
        # kernel_size : ukuran kernel filter yang digunakan
        # stride : jarak antar titik saat dilakukan convolution
        # padding : jumlah padding yang ditambahkan pada setiap sisi tensor sebelum dilakukan convolution

        # proses layer pertama nn.ConvTranspose2d dimulai dengan in_channels sebesar self.z_dim (ruang laten = 200 dimensi),
        # out_channels sebesar d_dim * 32 (16x32=512), kernel_size sebesar 4, stride sebesar 1, dan padding sebesar 0,
        # dari parameter tersebut menghasilkan 512 channel yang akan digunakan pada operasi layer berikutnya
        # setelah melakukan perhitungan dilakukan proses nn.BatchNorm2d pada output channel yang akan dipakai pada layer berikutnya
        # nn.BatchNorm2d adalah lapisan perhitungan untuk normalisasi data pada setiap batch,
        # dengan menghitung mean dan standard deviation dari setiap channel pada data input,
        # lalu data input dikalikan dengan standard deviation dan dikurangi dengan mean.
        # dilakukannya normalisasi untuk mengontrol ukuran data input agar tidak terlalu besar atau terlalu kecil, training stabil dan cepat converge
        # selanjutnya dilakukan proses aktivasi dengan nn.ReLU(true), yaitu untuk menambah non-linearitas pada model
        # nilai yang lebih kecil dari 0 akan diubah menjadi 0 dengan input dari output nn.ConvTranspose2 sebelumnya,
        # dan True berarti dilakukan secara inplace pada tensor tanpa membuat copy
        # selanjutnya akan diulang proses pada tiap layer dengan parameter yang berbeda hingga menghasilkan output dengan ukutan 128x128 3 channel (RGB)

        nn.ConvTranspose2d(z_dim, d_dim * 32, 4, 1, 0), # 4x4 (ch: 200, 512)
        nn.BatchNorm2d(d_dim * 32),
        nn.ReLU(True),

        # dilakukan proses transpose konvolusi ulang dengan in_channel merupakan output dari out_channel sebelumnya.
        # out_channel dideklarasikan dengan d_dim * 16 agar mendapatkan output 256 channel
        # parameter kernel_size, stride dan padding di isi dengan 4, 2, 1 yang bertujuan untuk mengubah ukuran height dan width menjadi 8x8
        # selanjutnya dilakukan normalisasi dan aktivasi seperti proses pada layer sebelumnya.

        nn.ConvTranspose2d(d_dim * 32, d_dim * 16, 4, 2, 1), # 8x8 (ch: 512, 256)
        nn.BatchNorm2d(d_dim * 16),
        nn.ReLU(True),

        # proses pada layer selanjutnya hampir sama dengan sebelumnya dengan mengubah parameter in dan out_channel agar menampilkan out 128
        # dan pada parameter kernel_size, stride dan padding sama dengan sebelumny karena dapat menghasilkan height width baru yaitu 16x16

        nn.ConvTranspose2d(d_dim * 16, d_dim * 8, 4, 2, 1), # 16x16 (ch: 256, 128)
        nn.BatchNorm2d(d_dim * 8),
        nn.ReLU(True),

        # proses pada layer selanjutnya hampir sama dengan sebelumnya dengan mengubah parameter in dan out_channel agar menampilkan out 64
        # dan pada parameter kernel_size, stride dan padding sama dengan sebelumny karena dapat menghasilkan height width baru yaitu 32x32

        nn.ConvTranspose2d(d_dim * 8, d_dim * 4, 4, 2, 1), # 32x32 (ch: 128, 64)
        nn.BatchNorm2d(d_dim * 4),
        nn.ReLU(True),

        # proses pada layer selanjutnya hampir sama dengan sebelumnya dengan mengubah parameter in dan out_channel agar menampilkan out 32
        # dan pada parameter kernel_size, stride dan padding sama dengan sebelumny karena dapat menghasilkan height width baru yaitu 64x64

        nn.ConvTranspose2d(d_dim * 4, d_dim * 2, 4, 2, 1), # 64x64 (ch: 64, 32)
        nn.BatchNorm2d(d_dim * 2),
        nn.ReLU(True),

        # proses pada layer terakhir sedikit berbeda dengan sebelumnya dimana pada in_channel masih menggunakan out_channel layer sebelumnya
        # sedangkan pada out_channel menggunakan nilai 3, hal ini bertujuan agar output channel sesuai dengan yang diinginkan yaitu RGB=3 chanel
        # dan pada parameter kernel_size, stride dan padding sama dengan sebelumny karena dapat menghasilkan height width baru yaitu 128x128
        # selanjutnya tidak dilakukan normalisasi maupun aktivasi melainkan fungsi nn.Tanh()
        # nn.Tanh() adalah fungsi aktivasi akhir dari layer pada generator. fungsi ini digunakan untuk membatasi nilai output dari generator pada rentang -1 sampai 1
        # agar output dari generator dapat diterima oleh fungsi loss yang digunakan dalam learning.
        # nn.Tanh() cocok digunakan karena menghasilkan output yang berdisitribusi normal

        nn.ConvTranspose2d(d_dim * 2, 3, 4, 2, 1), # 128x128 (ch: 32, 3)
        nn.Tanh() # produce result in the range of -1 & 1
    )

  # dideklarasikan fungsi forward yang merupakan fungsi forward propagation turunan dari nn.Module kelas generator
  # fungsi forward memiliki parameter noise, nantinya noise yang diinput akan diolah melalui proses Sequential sebelumnya
  # dilakukan pengubahan bentuk tensor noise dari awalnya (batch_size, z_dim) menjadi (batch_size, z_dim, 1, 1)
  # hal ini untuk menambahkan dimensi 1x1 untuk nantinya digunakan sebagi input pada proses konvolusi transpose nn.ConvTranspose2d di kelas generator
  # selanjutnya di return untuk menjalakan forward propagation pada generator dengan input x sebelumnya.

  def forward(self, noise):
    x = noise.view(len(noise), self.z_dim, 1, 1) # structure : 128 x 200 x 1 x 1
    return self.gen(x)

# dideklarasikan fungsi gen_noise yg digunakan untuk menghasilkan data acak yg nantinya digunakan sebagai input fungsi forward
# data acak ini disebut sebagai ruang laten (latent space) atau noise, fungsi ini memiliki parameter :
# num : jumlah data acak yang akan dihasilkan
# z_dim : jumlah dimensi pada ruang laten (latent space)
# device : perangkat yang digunakan untuk menghasilkan data acak, defaultnya adalah cuda (GPU)
# digunakan fungsi dari torch yaitu randn untuk menghasilkan data acak sesuai dengan jumlah dan dimensi yang sudah ditentukan,
# lalu data acak tersebut akan diubah menjadi tensor dan dikirim ke device (gpu)

def gen_noise(num, z_dim, device='cuda'):
  return torch.randn(num, z_dim, device=device) # 128 batch size x 200 dimensionality

In [7]:
# n : height & width, stride : amount of sliding, padding : amount of edges, ks : kernel size (3x3)
# nn.ConvTranspose2d : (n + 1) * stride - 2 * padding + ks
# nn.Conv2d : (n + 2 * pad - ks) // stride + 1

In [8]:
# critic model

# mendefinisikan kelas critic yang inherit nn.Module, bertugas mengevaluasi kevalidan dari suatu citra yang dihasilkan oleh generator

class Critic(nn.Module):

  # mendefinisikan fungsi init sebagai konstruktor kelas critic dengan hyperparametr self dan d_dim=16
  # d_dim memiliki value 16, merupakan jumlah output channel pada layer pertama kelas critic
  # output channels menentukan jumlah fitur yang diterima oleh layer setelah melalui proses konvolusi
  # menjalankan super.init untuk menjalankan inisialisasi dari kelas parent (nn.Module)

  def __init__(self, d_dim=16):
    super(Critic, self).__init__()

    # mendefinisikan arsitektur kelas critic menggunakan nn.Sequential yang memudahkan pembuatan arsitektur dari beberapa layer
    # ada 4 layer Conv2d yang digunakan untuk melakukan konvolusi pada citra dan 1 layer Linear yang digunakan untuk melakukan prediksi skor diskriminator

    self.crit = nn.Sequential(

        # Conv2d : in_channels, out_channels, kernel_size, stride=1, padding=0, d_dim=16
        # new width & height : (n + 2 * pad - ks) // stride +1

        # Conv2d merupakakan suatu layer konvolusi untuk melakukan operasi konvolusi antara filter dengan matriks gambar
        # Conv2d mengambil input gambar dan mengekstrak fitur yang mewakili gambar tersebut.
        # berfungsi untuk mengkombinasikan entri-entri dari setiap filter dengan data input,
        # dan memproduksi sebuah hasil konvolusi yang merupakan bagian dari layer baru. memiliki bbrp parametr yaitu:
        # in_channels : jumlah input channel, diawal dinisasi 3 karena input gambar adalah RGB
        # out_channels : jumlah output channel pada awal dinisasi dengan 16
        # kernel_size : ukuran kernel / filter dalam operasi konvolusi, diawal 4
        # stride : jarak antar titik saat dilakukan convolution
        # padding : jumlah padding yang ditambahkan pada setiap sisi tensor sebelum dilakukan convolution

        # pada layer pertama dilakukan Conv2d dengan parameternya dan menghasilkan 16 channel, serta width dan heigth baru yaitu 64x64
        # sebelum digunakan pada layer selanjutnya, out_channel dinormalisasi menggunakan fungsi nn.InstanceNorm2d
        # pada proses normalisasi, di hitung rata-rata dan deviasi standar dari setiap fitur (kanal) dalam set data,
        # lalu membagi setiap nilai dalam fitur dengan deviasi standar dan menambahkan rata-rata.
        # hal ini memastikan bahwa setiap fitur memiliki nilai yang sama besar dan membantu jaringan melakukan konvergensi lebih cepat.
        # selanjutnya dilakukan aktivasi LeakyReLU yang menerima hasil dari lapisan konvolisi sebelumnya
        # dan mengaktifkan fungsinya pada setiap pixel output. Fungsi ini membantu model untuk mempelajari fitur yang lebih rumit dalam gambar
        # pada LeakyRelu, dilakukan "leak" untuk membiarkan masuknya beberapa input negatif sebagai tingkat kerugian input negatif
        # pada kode ini, diisi value 0.2 yang brti jika input kurang dari 0, output akan menjadi 0.2 * input.
        # jika input lebih besar dari 0, output akan menjadi input itu sendiri.

        nn.Conv2d(3, d_dim, 4, 2, 1), # (128 + 2 * 1 - 4) // 2 +1 = 126 // 2 = 63 + 1 = 64 (ch: 3, 16) # 64x64
        nn.InstanceNorm2d(d_dim),
        nn.LeakyReLU(0.2),

        # pada layer selanjutnya dilakukan proses yang sama yaitu convolusi dengan Conv2d, normalisasi dengan Instance dan aktivasi dengan LeakyRelu
        # digunakan nilai output channel pada layer sebelumnya sebagai input channel dan menghasilkan wxh 32x32 serta out_channel 32.
        # serta output channel dinaikan membantu untuk memperkuat model dan memberikan lebih banyak pilihan untuk mempelajari detail dari gambar.
        # selain itu ini membantu memperkuat representasi dari setiap lapisan dan memastikan model memiliki informasi yang kuat untuk prediksi
        # pada tiap layer ukuran terus menurun sedangkan channel terus naik hal ini karena model mempertimbangkan informasi pada skala besar,
        # tidak hanya dari sebagian informasi saja
        # sedangkan channel terus naik yg brrti model ditiap layer mempelajari fitur yang lebih kompleks dan abstrak dari citra asli.

        nn.Conv2d(d_dim, d_dim*2, 4, 2, 1), # 32x32 (ch: 16, 32)
        nn.InstanceNorm2d(d_dim*2),
        nn.LeakyReLU(0.2),

        # dilakukan proses yang sama layer dengan out_channel layer sebelumnya menjadi in channel serta menaikkan out channel menjadi 64
        # menghasilkan wxh 16x16 dan out channel 64, dilakukan normalisasi pda out channel sebelum digunakan pada layer selanjutnya dan aktivasi juga.

        nn.Conv2d(d_dim*2, d_dim*4, 4, 2, 1), # 16x16 (ch: 32, 64)
        nn.InstanceNorm2d(d_dim*4),
        nn.LeakyReLU(0.2),

        # masih sama proses yang dilakukan hanya dirubah pada parameter in dan out channel

        nn.Conv2d(d_dim*4, d_dim*8, 4, 2, 1), # 8x8 (ch: 64, 128)
        nn.InstanceNorm2d(d_dim*8),
        nn.LeakyReLU(0.2),
        nn.Conv2d(d_dim*8, d_dim*16, 4, 2, 1), # 4x4 (ch: 128, 256)
        nn.InstanceNorm2d(d_dim*16),
        nn.LeakyReLU(0.2),

        # pada layer terakhir memiliki parameter yang berbeda yaitu out channel 1 serta stride 1 dan padding 0
        # out channel 1 karena untuk menentukan skor validitas gambar pada generator,
        # output dari layer ini harus berupa skala satu dimensi karena merupakan skor validitas gambar dari generator.
        # sedangkan stride 1 padding 0 untuk memastikan bahwa ukuran output dari layer ini sama dengan ukuran input,
        # sehingga mudah untuk dicompare dengan target skor validitas gambar.

        nn.Conv2d(d_dim*16, 1, 4, 1, 0) # (4 + 2 * 0 - 4) // 1 +1 = 1x1 (ch: 256, 1)
    )

  # didefinisikan fungsi forward dengan parameter image
  # fungsi forward digunakan untuk melakukan forward propagation dari suatu input gambar melalui arsitektur yg didefinisikan sebelumnya.
  # setiap layer yang ada dalam objek "self.main" akan diterapkan secara berurutan pada input "image"
  # setelah melalui semua layer, output dari fungsi forward adalah tensor yang mewakili nilai prediksi dari kelas critic pada input gambar tersebut
  # skor kevalidan gambar sebagai real atau buatan digunakan sebagai evaluasi

  def forward(self, image):
    # image : 128 (batch) x 3 (channel) x 128 x 128 (height and width)
    crit_pred = self.crit(image) # 128 x 1 x 1 x 1
    return crit_pred.view(len(crit_pred), -1) # 128 x 1

In [9]:
# optional, init weight in different way
def init_weights(m):
  if isintance(m, nn.Conv2d) or isintance(m, nn.ConvTranspose2d):
    torch.nn.init.normal_(m.weight, 0.0, 0.02)
    torch.nn.init.constant_(m.bias, 0)
  if isintance(m, nn.BatchNorm2d):
    torch.nn.init.normal_(m.weight, 0.0, 0.02)
    torch.nn.init.constant_(m.bias, 0)
# gen = gen.apply(init_weights)
# crit = crit.apply(init_weights)

In [10]:
from google.colab import drive
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

# Path ke file zip di Google Drive
zip_file_path = '/content/drive/MyDrive/batik_coba.zip'

# Path tujuan untuk mengekstrak dataset
extracted_path = '/content/batik'  # Ubah sesuai kebutuhan Anda

# Mengekstrak file zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)


Mounted at /content/drive


In [11]:
from google.colab import drive
import os
import PIL
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader

# Mount Google Drive
drive.mount('/content/drive')

# Definisi kelas Dataset
class CustomDataset(Dataset):
    def __init__(self, path, size=128, limit=10000):
        self.sizes = [size, size]
        items, labels = [], []
        for data in os.listdir(path)[:limit]:
            # path: '/content/drive/MyDrive/path/to/your/dataset'
            # data: 129880.JPG
            item = os.path.join(path, data)
            items.append(item)
            labels.append(data)
        self.items = items
        self.labels = labels

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        data = PIL.Image.open(self.items[idx]).convert('RGB') # 178x128
        data = np.asarray(torchvision.transforms.Resize(self.sizes)(data)) # 128x128x3 (resized)
        data = np.transpose(data, (2, 0, 1)).astype(np.float32, copy=False) # 3x128x128 from 0 to 255
        data = torch.from_numpy(data).div(255) # from 0 to 1
        return data, self.labels[idx]

# Path ke folder dataset di Google Drive
data_path = '/content/batik/vz7pzt2grf-1'

# Inisialisasi objek dataset
ds = CustomDataset(data_path, size=128, limit=10000)

# DataLoader
batch_size = 64
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True)

# Model generator dan kritikus
gen = Generator(z_dim).to(device)
crit = Critic().to(device)

# Optimizer
lr = 0.0002
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.9))
crit_opt = torch.optim.Adam(crit.parameters(), lr=lr, betas=(0.5, 0.9))

# Inisialisasi
if wandbact == 1:
    wandb.watch(gen, log_freq=100)
    wandb.watch(crit, log_freq=100)

# Menampilkan contoh data dari DataLoader
x, y = next(iter(dataloader))
show(x)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# gradient penalty calculation
def get_gp(real, fake, crit, alpha, gamma=10):
  mix_images = real * alpha + fake * (1-alpha) # 128x3x128x128
  mix_scores = crit(mix_images) # 128x1
  gradient = torch.autograd.grad(
      inputs = mix_images,
      outputs = mix_scores,
      grad_outputs = torch.ones_like(mix_scores),
      retain_graph = True,
      create_graph = True
  )[0] #128x3x128x128
  gradient = gradient.view(len(gradient), -1) #128x49152
  gradient_norm = gradient.norm(2, dim=1)
  gp = gamma * ((gradient_norm-1) ** 2).mean()
  return gp

In [ ]:
# save & load checkpoints
root_path = './data/'
def save_checkpoint(name):
  #gen
  torch.save({
      'epoch': epoch,
      'model_state_dict': gen.state_dict(),
      'optimizer_state_dict': gen_opt.state_dict()
  }, f"{root_path}G-{name}.pkl")
  #crit
  torch.save({
      'epoch': epoch,
      'model_state_dict': crit.state_dict(),
      'optimizer_state_dict': crit_opt.state_dict()
  }, f"{root_path}C-{name}.pkl")
  # print("checkpoint saved successfully")

def load_checkpoint(name):
  #gen
  checkpoint = torch.load(f"{root_path}G-{name}.pkl")
  gen.load_state_dict(checkpoint['model_state_dict'])
  gen_opt.load_state_dict(checkpoint['optimizer_state_dict'])
  #crit
  checkpoint = torch.load(f"{root_path}C-{name}.pkl")
  crit.load_state_dict(checkpoint['model_state_dict'])
  crit_opt.load_state_dict(checkpoint['optimizer_state_dict'])
  print("checkpoint loaded successfully")


In [ ]:
gen

In [ ]:
# epoch=1
# save_checkpoint("test")

In [ ]:
# training loop
for epoch in range(n_epochs):
  for real, _ in tqdm(dataloader):
    cur_bs = len(real) #128
    real = real.to(device)
    # critic
    mean_crit_loss = 0
    for _ in range(crit_cycles):
      crit_opt.zero_grad()
      noise = gen_noise(cur_bs, z_dim)
      fake = gen(noise)
      crit_fake_pred = crit(fake.detach())
      crit_real_pred = crit(real)
      # calculate gradient penalty
      alpha = torch.rand(len(real), 1, 1, 1, device=device, requires_grad=True) # 128x1x1x1
      gp = get_gp(real, fake.detach(), crit, alpha)
      crit_loss = crit_fake_pred.mean() - crit_real_pred.mean() + gp
      mean_crit_loss += crit_loss.item() / crit_cycles
      crit_loss.backward(retain_graph=True)
      crit_opt.step()
    crit_losses += [mean_crit_loss]
    # generator
    gen_opt.zero_grad()
    noise = gen_noise(cur_bs, z_dim)
    fake = gen(noise)
    crit_fake_pred = crit(fake)
    gen_loss = -crit_fake_pred.mean()
    gen_loss.backward()
    gen_opt.step()
    gen_losses += [gen_loss.item()]

    # stats
    if(wandbact==1):
      wandb.log({'Epoch': epoch, 'Step': cur_step, 'Critic loss': mean_crit_loss, 'Gen loss': gen_loss})

    if cur_step % save_step and cur_step > 0:
      print("saving checkpoint", cur_step, save_step)
      save_checkpoint("latest")

    if(cur_step % show_step == 0 and cur_step > 0):
      show(fake, wandbactive=1, name='Fake')
      show(real, wandbactive=1, name='Real')

      gen_mean = sum(gen_losses[-show_step:]) / show_step
      crit_mean = sum(crit_losses[-show_step:]) / show_step
      print(f"Epoch: {epoch}: Step {cur_step}: Generator loss {gen_mean}, Critic loss {crit_mean}")

      plt.plot(
          range(len(gen_losses)),
          torch.Tensor(gen_losses),
          label = "Generator loss"
      )
      plt.plot(
          range(len(crit_losses)),
          torch.Tensor(crit_losses),
          label = "Critic loss"
      )

      plt.ylim(-1000, 1000)
      plt.legend()
      plt.show()

    cur_step += 1


In [ ]:
# generate
noise = gen_noise(batch_size, z_dim)
fake = gen(noise)
show(fake)

In [ ]:
plt.imshow(fake[4].detach().cpu().permute(1,2,0).squeeze().clip(0,1))

In [ ]:
# morphing, interpolating between point in latent space
from mpl_toolkits.axes_grid1 import ImageGrid

# MORPHING, interpolation between points in latent space
gen_set=[]
z_shape=[1,200,1,1]
rows=4
steps=17

for i in range(rows):
  z1,z2 = torch.randn(z_shape), torch.randn(z_shape)
  for alpha in np.linspace(0,1,steps):
    z=alpha*z1 + (1-alpha)*z2
    res=gen(z.cuda())[0]
    gen_set.append(res)

fig = plt.figure(figsize=(25,11))
grid=ImageGrid(fig, 111, nrows_ncols=(rows,steps), axes_pad=0.1)

for ax , img in zip (grid, gen_set):
  ax.axis('off')
  res=img.cpu().detach().permute(1,2,0)
  res=res-res.min()
  res=res/(res.max()-res.min())
  ax.imshow(res.clip(0,1.0))

plt.show()